In [1]:
import glob
import numpy as np
import copy

filepaths_to_be_wrapped = []
for i in range(1, 2):
    for file_name in set(glob.glob(str(i) + 'run/*.pdb')) - set(glob.glob(str(i) + 'run/*wrapped*')):
        filepaths_to_be_wrapped.append(file_name)
print(filepaths_to_be_wrapped)


['1run\\snapshot_2.pdb', '1run\\118000_checkpoint.pdb']


In [2]:
for file_name in filepaths_to_be_wrapped:
    temp_newfilename = file_name.split('.')
    newfilename = temp_newfilename[0] + '_wrapped.' + temp_newfilename[1]
    with open(file_name, 'r') as f:
        water_index_list, acrolein_index_list = [], []
        lines = f.readlines()
        for i in range(len(lines)):
            if 'CRYST1' in lines[i]:
                need = lines[i].split()
                x_bound, y_bound, z_boudn = float(need[1]), float(need[2]), float(need[3])
            if 'ATOM' in lines[i]:
                if ' O ' in lines[i]:
                    if ' H ' in lines[i+1]:
                        water_index_list.append(i)
                    if ' C ' in lines[i+1]:
                        acrolein_index_list.append(i)
            f.close()

In [55]:
def subtract_atom(molecule, index, bounds):
    for i in range(len(molecule)):
        molecule[i][index] = round(float(molecule[i][index]) - bounds, 3)
    return molecule

def add_atom(molecule, index, bounds):
    for i in range(len(molecule)):
        molecule[i][index] = round(float(molecule[i][index]) + bounds, 3)
    return molecule

def atom_together(atom_1, atom_2):
    dist0 = atom_1[0] - atom_2[0]
    dist1 = atom_1[1] - atom_2[1]
    dist2 = atom_1[2] - atom_2[2]
    if abs(dist0) > 5 or abs(dist1) > 5 or abs(dist2) > 5:
        return False
    else:
        return True

def dist(atom_dim_1, atom_dim_2):
    return round(atom_dim_1 - atom_dim_2, 3)

def align(atom_a, atom_b, atom_c):
    xab = dist(atom_dim_1 = atom_a[0], atom_dim_2 = atom_b[0])
    yab = dist(atom_dim_1 = atom_a[1], atom_dim_2 = atom_b[1])
    zab = dist(atom_dim_1 = atom_a[2], atom_dim_2 = atom_b[2])
    xac = dist(atom_dim_1 = atom_a[0], atom_dim_2 = atom_c[0])
    yac = dist(atom_dim_1 = atom_a[1], atom_dim_2 = atom_c[1])
    zac = dist(atom_dim_1 = atom_a[2], atom_dim_2 = atom_c[2])
    xbc = dist(atom_dim_1 = atom_b[0], atom_dim_2 = atom_c[0])
    ybc = dist(atom_dim_1 = atom_b[1], atom_dim_2 = atom_c[1])
    zbc = dist(atom_dim_1 = atom_b[2], atom_dim_2 = atom_c[2])
    print(xab, xac, xbc)
    print(yab, yac, ybc)
    print(zab, zac, zbc)
    

def check_together(atom_a, atom_b, atom_c):
    atom_a_atom_b_together = atom_together(atom_1 = atom_a, atom_2 = atom_b)
    atom_a_atom_c_together = atom_together(atom_1 = atom_a, atom_2 = atom_c)
    atom_b_atom_c_together = atom_together(atom_1 = atom_b, atom_2 = atom_c)
    if atom_a_atom_b_together == False or atom_a_atom_c_together == False or atom_b_atom_c_together == False:
        return False
    else:
        return True

In [56]:
molecules = []
falses = []
trues = []
translated_molecules = []
for i in range(len(water_index_list)):
    molecule = []
    out_of_bounds_list = []
    for x in range(3):
        atom = lines[water_index_list[i] + x].split()
        atom = [float(atom[u]) for u in range(5, 8)]
        molecule.append(atom)
    molecules.append(molecule)
old_molecules = copy.deepcopy(molecules)
together_molecules = copy.deepcopy(molecules)
for x in range(len(molecules)):
    together = check_together(atom_a = molecules[x][0], atom_b = molecules[x][1], atom_c = molecules[x][2])
    if together == False:
        together_molecules[x] = align(atom_a = molecules[x][0], atom_b = molecules[x][1], atom_c = molecules[x][2])
        falses.append('False')
    else:
        together_molecules[x] = molecules[x]
        trues.append('True')
    for y in range(len(molecules[x][0])):
        coordinate = molecules[x][0][y]
        if coordinate < 0:
            #print('old_molecule')
            #print(molecules[x])
            new_molecule = add_atom(molecule = molecules[x], index = y, bounds = x_bound)
            #print('new_molecule')
            #print(new_molecule)
        elif coordinate > x_bound:
            #print('old_molecule')
            new_molecule = subtract_atom(molecule = molecules[x], index = y, bounds = x_bound)
            #print('new_molecule')
            #print(new_molecule)
        else:
            new_molecule = molecules[x]
             
    translated_molecules.append(new_molecule)
#print(len(old_molecules))
#print(len(translated_molecules))
array_old_molecules= np.array(old_molecules)
array_translated_molecules = np.array(translated_molecules)
#print(array_translated_molecules - array_old_molecules)
#print('ayy')

print(len(falses))
print(len(trues))

-0.045 -0.294 -0.249
-24.466 -0.532 23.934
0.88 -0.597 -1.477
0.577 -0.846 -1.423
-0.451 24.402 24.853
-25.04 -24.881 0.159
0.081 0.7 0.619
0.826 0.082 -0.744
0.478 -25.001 -25.479
0.307 0.707 0.4
0.834 -0.007 -0.841
23.891 0.717 -23.174
0.833 -25.085 -25.918
-0.338 -0.534 -0.196
-0.434 -0.38 0.054
-0.731 -0.461 0.27
-23.959 -0.732 23.227
0.503 -0.412 -0.915
-0.817 0.576 1.393
-0.112 -0.59 -0.478
23.902 -0.451 -24.353
-0.443 -0.667 -0.224
-0.779 0.578 1.357
24.77 -0.381 -25.151
0.663 -0.439 -1.102
24.251 25.204 0.953
-0.744 -0.259 0.485
24.55 0.626 -23.924
-0.134 0.186 0.32
0.845 -0.687 -1.532
-0.633 -23.561 -22.928
0.486 0.398 -0.088
-0.596 -0.154 0.442
-0.833 0.544 1.377
0.557 0.532 -0.025
-0.156 24.932 25.088
0.502 -24.384 -24.886
0.866 -0.521 -1.387
-0.009 0.778 0.787
-0.915 24.603 25.518
-0.006 0.82 0.826
0.299 -0.407 -0.706
0.808 -0.629 -1.437
0.005 -25.072 -25.077
0.331 -0.019 -0.35
-0.767 25.125 25.892
0.24 -0.214 -0.454
0.532 0.595 0.063
0.319 0.588 0.269
-23.615 0.168 23.783


In [41]:
with open(newfilename, 'w') as f:
    new_lines = copy.deepcopy(lines)
    for i in range(len(lines)):
        new_lines[i] = lines[i].split()
    counter = 0

    for i in range(len(lines)):
        

        if i in water_index_list:
            new_lines[i][5] = str(translated_molecules[counter][0][0])
            new_lines[i][6] = str(translated_molecules[counter][0][1])
            new_lines[i][7] = str(translated_molecules[counter][0][2])
            new_lines[i+1][5] = str(translated_molecules[counter][1][0])
            new_lines[i+1][6] = str(translated_molecules[counter][1][1])
            new_lines[i+1][7] = str(translated_molecules[counter][1][2])
            new_lines[i+2][5] = str(translated_molecules[counter][2][0])
            new_lines[i+2][6] = str(translated_molecules[counter][2][1])
            new_lines[i+2][7] = str(translated_molecules[counter][2][2])
            counter += 1
        else:
            pass
    f.write(lines[0])
    f.write(lines[1])
    for i in range(2, len(new_lines)-1):
        f.write('{:<6}'.format(new_lines[i][0]) + '{:>5}'.format(new_lines[i][1]) + '{:>5}'.format(new_lines[i][2]) + '{:>4}'.format(new_lines[i][3]) + '{:>6}'.format(new_lines[i][4]) + '{:>11}'.format(new_lines[i][5]) + '{:>8}'.format(new_lines[i][6]) + '{:>8}'.format(new_lines[i][7]) + '{:>6}'.format(new_lines[i][8]) + '{:>6}'.format(new_lines[i][9]) + '{:>12}'.format(new_lines[i][10]) + '\n')

    f.write(lines[-1])